# Zurich Instruments LabOne Python API Example
# Use the Impedance Module

Demonstrate how the LabOne Impedance Module can be used to do a user
compensation (calibration).

Requirements:

* LabOne Version >= 22.08
* Instruments:
    1 x Instrument with IA option

In [ ]:
from zhinst.core import ziDAQServer

device_id = "dev3519" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection or when MFLI/MFIA is connected to a remote Data Server.
#interface = "USB" # For all instruments connected to the host computer via USB except MFLI/MFIA.
#interface = "PCIe" # For MFLI/MFIA devices in case the Data Server runs on the device.

server_host = "localhost"
server_port = 8004
api_level = 6 # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

### Instrument settings

In [ ]:
settings = []
device_imp_path = f"{device_id}/imps/0/"
settings.append((device_imp_path + "enable", 1))
settings.append((device_imp_path + "mode", 0))
settings.append((device_imp_path + "auto/output", 1))
settings.append((device_imp_path + "auto/bw", 1))
settings.append((device_imp_path + "auto/inputrange", 1))
settings.append((device_imp_path + "freq", 1000))
settings.append((device_imp_path + "output/amplitude", 0.3))
settings.append((device_imp_path + "output/range", 1))
settings.append((device_imp_path + "model", 0))
daq.set(settings)

### User compensation setup

In [ ]:
module = daq.impedanceModule()

# Start the module
module.execute()

module.set("/device", device_id)
# Set frequency range from 1000 to 5000000 with 20 number of samples
module.set("/freq/start", 1000)
module.set("/freq/stop", 5000000)
module.set("/freq/samplecount", 20)
# Disabling the validation for demonstration prupose.
# (this allows everything to pass, in case the short is much higher than 0 Ohm.)
module.set("/validation", 0)
# Short only mode with additional open step
# 1 = Short
# 4 = Load
# 5 = Short Load
# 8 = Load Load Load
# Note that the additional open step can be added to all of the options above
# except the Load Load Load option.
module.set("/mode", 1)
module.set("/openstep", 1)

Define the helper functions to track the progress of the impedance module.

In [ ]:
from zhinst.core import ImpedanceModule
import typing as t
import time

def finished_step(module: ImpedanceModule, step: t.Optional[int] = None) -> bool:
    """Check if the calibration or a step of it is finished.

    Args:
        step: Calibration step. If not None this function checks if the
            specified step is finished. Otherwise it checks if the
            whole calibration is done.

    Returns:
        Flag if the calibration or a step is finished.
    """
    if step is None:
        return module.getInt("/status") == module.getInt("/expectedstatus")
    return module.getInt("/status") & (1 << step)

def wait_done(
    module: ImpedanceModule,
    step: t.Optional[int] = None,
    *,
    timeout: float = 20.0,
    sleep_time: float = 0.5,
) -> None:
    """Waits until the specified compensation step is complete.

    Args:
        step: The compensation step to wait for completion.
        timeout: The maximum waiting time in seconds for the compensation
            to complete (default: 20).
        sleep_time: Time in seconds to wait between
            requesting the state. (default: 0.5)

    Raises:
        TimeoutError: The compensation is not completed before timeout.
    """
    start_time = time.time()
    while (
        start_time + timeout >= time.time()
        and module.getInt("/calibrate")
        and not finished_step(module, step)
    ):
        print(f"Progress: {(module.progress()[0] * 100):.1f}%")
        time.sleep(sleep_time)
    if module.progress()[0] < 1:
        raise TimeoutError("Impedance module timed out.")
    if not finished_step(module, step):
        if step is None:
            raise RuntimeError(
                "Impedance module did not reach the status "
                f"{module.getInt('/expectedstatus')} that "
                "corresponds to a full compensation. "
                f"(current status: {module.getInt('/status')})"
            )
        raise RuntimeError(
            f"Impedance module did not finish the requested step {step}. "
            f"(current status: {module.getInt('/status')})"
        )

### Short compensation

The first compensation step is the `Short` step.

In [ ]:
step = 0
module.set("/step", step)
module.set("/calibrate", 1)
wait_done(module, step=step, sleep_time=2)

Log messages from the impedance module during the first (short) calibration step.

In [ ]:
import html
print(html.unescape(module.getString("/message")))

### Open compensation
Please change from short into open now.

In [ ]:
step = 1
module.set("/step", step)
module.set("/calibrate", 1)
wait_done(module, step=step, sleep_time=2)

Log messages from the impedance module during the second (open) calibration step.

In [ ]:
import html
print(html.unescape(module.getString("/message")))

### Save and upload compensation file

In [ ]:
module.set("/todevice", 1)

In [ ]:
from pathlib import Path

target_path = Path("testcal")
module.set("/directory", str(target_path.parent.absolute()))
module.set("/filename", target_path.stem)
module.set("/save", 1)

# Use this to apply the just finished short-open compensation
# module.set("/load", 1)